In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import date
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn import preprocessing
from sklearn.externals import joblib

import warnings
warnings.filterwarnings("ignore")
import math
import re
import datetime

from google.colab import drive

In [0]:
def add_col_brand(users):
  users['brand'] = devices['brand']
  mapping_dict_brand = {'Apple': 0, 'Android': 1, 'Unknown': float('NaN')}
  users['brand'] = users['brand'].map(mapping_dict_brand) 
  return users

def mapping_data(users):  
  mapping_dict_plan = {'STANDARD': 0, 'SILVER': 1, 'GOLD': 1}
  users['plan'] = users['plan'].map(mapping_dict_plan)
  return users

def add_col_age(users):
  now = datetime.datetime.now()
  users['age'] = users['birth_year'].apply(lambda x: now.year - x)
  return users

def user_id_to_int(df):
  df['user_id'] = df['user_id'].apply(lambda x: int(x.split("_")[1]))
  return df

def add_inbound_outbound(users,users_trans):
  inbound_list = [0] * users.shape[0]
  outbound_list = [0] * users.shape[0]
  inbound_outbound_ratio_list  = [0] * users.shape[0]

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['direction'] == 'INBOUND':
      inbound_list[users_trans_row['user_id']] = inbound_list[users_trans_row['user_id']] + 1
    elif users_trans_row['direction'] == 'OUTBOUND':
      outbound_list[users_trans_row['user_id']] = outbound_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    inbound_outbound_total = inbound_list[i] + outbound_list[i]
    if (inbound_outbound_total == 0):
      inbound_outbound_ratio_list[i] = 0
    else:  
      inbound_outbound_ratio_list[i] = inbound_list[i] / inbound_outbound_total

  
  inbound_ser = pd.Series(inbound_list)
  outbound_ser = pd.Series(outbound_list)
  inbound_outbound_ratio_ser = pd.Series(inbound_outbound_ratio_list)
  users['inbound_transactions'] = inbound_ser
  users['outbound_transactions'] = outbound_ser
  users['inbound_outbound_ratio'] = inbound_outbound_ratio_ser
  return users

def add_completed_to_failed_trans_ratio(users,users_trans):
  completed_list = [0] * users.shape[0]
  failed_list = [0] * users.shape[0]
  completed_failed_ratio_list  = [0] * users.shape[0]

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['transactions_state'] == 'COMPLETED':
      completed_list[users_trans_row['user_id']] = completed_list[users_trans_row['user_id']] + 1
    elif users_trans_row['transactions_state'] == 'FAILED':
      failed_list[users_trans_row['user_id']] = failed_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    completed_failed_total = completed_list[i] + failed_list[i]
    if (completed_failed_total == 0):
      completed_failed_ratio_list[i] = 0
    else:  
      completed_failed_ratio_list[i] = completed_list[i] / completed_failed_total

  
  completed_ser = pd.Series(completed_list)
  failed_ser = pd.Series(failed_list)
  completed_failed_ratio_ser = pd.Series(completed_failed_ratio_list)
  users['completed_transactions'] = completed_ser
  users['failed_transactions'] = failed_ser
  users['completed_failed_ratio'] = completed_failed_ratio_ser
  return users

def add_average_amount_per_transaction(users,users_trans):
  avg_amount_list = [0] * len(users)
  
  for idx_users_trans, users_trans_row in users_trans.iterrows():
    avg_amount_list[users_trans_row['user_id']] = avg_amount_list[users_trans_row['user_id']] + users_trans_row['amount_usd'] 

  for idx_user, users_row in users.iterrows():
    avg_amount_list[idx_user] = avg_amount_list[idx_user] / users_row['transactions_count']
      
  avg_amount_ser = pd.Series(avg_amount_list)
  users['average_amount_per_transaction'] = avg_amount_ser
  return users

def add_abroad_to_non_abroad(users, users_trans):
  abroad_trans_list = []
  non_abroad_trans_list = []
  abr_to_non_abr_ratio = []

  for i in range(len(users)):
    abroad_trans_list.append(0)
    non_abroad_trans_list.append(0)
    abr_to_non_abr_ratio.append(0)

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['country'] != users_trans_row['ea_merchant_country'] and users_trans_row['ea_merchant_country'] != float('NaN'):
      abroad_trans_list[users_trans_row['user_id']] = abroad_trans_list[users_trans_row['user_id']] + 1
    else:
      non_abroad_trans_list[users_trans_row['user_id']] = non_abroad_trans_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    abr_to_non_abr_ratio[i] = abroad_trans_list[i] / (abroad_trans_list[i] + non_abroad_trans_list[i])

  abroad_trans_ser = pd.Series(abroad_trans_list)
  non_abroad_trans_ser = pd.Series(non_abroad_trans_list)
  abr_to_non_abr_ratio_ser = pd.Series(abr_to_non_abr_ratio)

  users['abroad_transactions'] = abroad_trans_ser
  users['non_abroad_transactions'] = non_abroad_trans_ser
  users['abroad_to_non_abroad_ratio'] = abr_to_non_abr_ratio_ser

  return users

def add_cardholder_presence(users, users_trans):
  presence_list = []
  non_presence_list = []
  pres_to_non_pres_ratio = []

  for i in range(len(users)):
    presence_list.append(0)
    non_presence_list.append(0)
    pres_to_non_pres_ratio.append(0)

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['ea_cardholderpresence'] == 'TRUE':
      presence_list[users_trans_row['user_id']] = presence_list[users_trans_row['user_id']] + 1
    elif users_trans_row['ea_cardholderpresence'] == 'FALSE':
      non_presence_list[users_trans_row['user_id']] = non_presence_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    total =  (presence_list[i] + non_presence_list[i])
    if(total!=0):
      pres_to_non_pres_ratio[i] = presence_list[i] / total

  pres_to_non_pres_ratio_ser = pd.Series(pres_to_non_pres_ratio)
  users['presence_to_non_presence_ratio'] = pres_to_non_pres_ratio_ser

  return users

def add_largest_trans_amount(users, users_trans):
  largest_trans_list = []

  for i in range(len(users)):
    largest_trans_list.append(0)

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if(largest_trans_list[users_trans_row['user_id']] != float('NaN')):
      if users_trans_row['amount_usd'] > int(largest_trans_list[users_trans_row['user_id']]):
        largest_trans_list[users_trans_row['user_id']] = users_trans_row['amount_usd'] 

  largest_trans_ser = pd.Series(largest_trans_list)
  users['largest_transaction_amount'] = largest_trans_ser

  return users  

def add_months_subscribed(users):
  now = datetime.datetime.now()
  months_subscribed_list = [0] * users.shape[0]

  for idx_users, users_row in users.iterrows():
    date_split = users_row['created_date'].split("-")
    year_created = int(date_split[0])
    month_created = int(date_split[1])
    if now.year == year_created:
      total_months = now.month - month_created
    else:
      diff = now.year - year_created  
      total_months = now.month + ((diff * 12) - month_created)
    
    months_subscribed_list[idx_users] = total_months

  months_subscribed_ser = pd.Series(months_subscribed_list)
  users['months_subscribed'] = months_subscribed_ser

  return users

def add_transaction_count(users,users_trans):
  trans_count_by_user = users_trans.groupby(['user_id']).count()
  users['transactions_count'] = trans_count_by_user.iloc[:,0] 
  return users


def add_trans_total_amount(users,users_trans):
  trans_sum_by_user = users_trans.groupby(['user_id']).sum()
  users['transactions_total_amount'] = trans_sum_by_user['amount_usd'] 
  return users


def convert_country_code(users, country_codes_file):
  dictionary = {}
  with open(country_codes_file) as f:
      for line in f:
        country, alpha2_code, alpha3_code, numeric = line.split('|')
        dictionary[alpha2_code] = alpha3_code

  users['country'] = users['country'].map(dictionary)
  return users


def add_mcc(users, users_trans):

  users_trans['ea_merchant_mcc'] = users_trans['ea_merchant_mcc'].fillna(0)

  size = users.shape[0]
  all_mcc_lists = []
  all_mcc_count_lists = []
  mcc_categories = [(1, 1499), (1500, 2999), (3000, 3999), (4000, 4799), (4800, 4999), (5000, 5599), (5600, 5699), (5700, 7299), (7300, 7999), (8000, 8999), (9000, 9999)]

  for list in range(len(mcc_categories)):
    mcc_list = [0] * size
    mcc_count = [0] * size

    all_mcc_lists.append(mcc_list)
    all_mcc_count_lists.append(mcc_count)


  for idx_users_trans, users_trans_row in users_trans.iterrows():
    user_mcc = users_trans_row['ea_merchant_mcc']
    for index, category in enumerate(mcc_categories):
      if(user_mcc >= category[0] and user_mcc <= category[1]):
        all_mcc_lists[index][users_trans_row['user_id']] = all_mcc_lists[index][users_trans_row['user_id']] + users_trans_row['amount_usd']
        all_mcc_count_lists[index][users_trans_row['user_id']] = all_mcc_count_lists[index][users_trans_row['user_id']] + 1


  for idx_mcc_cat, mcc_category in enumerate(all_mcc_count_lists):
    for idx_mcc_user, mcc_user in enumerate(mcc_category):
      if(mcc_user != 0):
        all_mcc_lists[idx_mcc_cat][idx_mcc_user] = all_mcc_lists[idx_mcc_cat][idx_mcc_user] / mcc_user
      else: 
        all_mcc_lists[idx_mcc_cat][idx_mcc_user] = 0

  for mcc_category in range(len(all_mcc_lists)):
    users['mcc_'+str(mcc_category)] = pd.Series(all_mcc_lists[mcc_category])

  
  return users


def cardholder_pres_fillna(presence):
  
  if presence == 'UNKNOWN':
    return 'FALSE'
  else:
    return presence


def cardholder_fillna(df):
  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].fillna('UNKNOWN')

  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].apply(cardholder_pres_fillna)
  return df


def remove_outliers(series):
  x = series
  UPPERBOUND, LOWERBOUND = np.percentile(x, [1,99])
  y = np.clip(x, UPPERBOUND, LOWERBOUND)
  return pd.Series(y)


def normalize(series):
  scaler = preprocessing.MinMaxScaler()
  column_as_array = series.to_numpy().reshape(-1,1)
  scaled = scaler.fit_transform(column_as_array)
  scaled_df = pd.DataFrame(scaled)

  joblib.dump(scaler, project_dir + 'auxiliary_data/scalers/MinMax_' + series.name + '.save')

  return scaled_df[0]


def standardize(series):
  scaler = preprocessing.StandardScaler()
  column_as_array = series.to_numpy().reshape(-1,1)
  scaled = scaler.fit_transform(column_as_array)
  scaled_df = pd.DataFrame(scaled)

  joblib.dump(scaler, project_dir + 'auxiliary_data/scalers/Stand_' + series.name + '.save')

  return scaled_df[0]


def extract_trans_features(users, transactions):

    user_trans = pd.merge(transactions, users,on='user_id',how='left')

    inbound_per_user = pd.DataFrame(transactions[transactions['direction']== 'INBOUND'].groupby(['user_id'])['direction'].size().reset_index(name='inbound_transactions'))
    users = pd.merge(inbound_per_user, users, on='user_id', how='outer')
    
    outbound_per_user = pd.DataFrame(transactions[transactions['direction']== 'OUTBOUND'].groupby(['user_id'])['direction'].size().reset_index(name='outbound_transactions'))
    users = pd.merge(outbound_per_user, users, on='user_id', how='outer')



    completed_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'COMPLETED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='completed_transactions'))
    users = pd.merge(completed_per_user, users, on='user_id', how='outer')

    declined_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'DECLINED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='declined_transactions'))
    users = pd.merge(declined_per_user, users, on='user_id', how='outer')

    reverted_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'REVERTED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='reverted_transactions'))
    users = pd.merge(reverted_per_user, users, on='user_id', how='outer')

    failed_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'FAILED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='failed_transactions'))
    users = pd.merge(failed_per_user, users, on='user_id', how='outer')

    pending_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'PENDING'].groupby(['user_id'])['transactions_state'].size().reset_index(name='pending_transactions'))
    users = pd.merge(pending_per_user, users, on='user_id', how='outer')

    cancelled_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'CANCELLED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='cancelled_transactions'))
    users = pd.merge(cancelled_per_user, users, on='user_id', how='outer')

    #number of transactions per user
    trans_per_user = pd.DataFrame(transactions.groupby(['user_id'])['transactions_state'].size().reset_index(name='TransPerUser'))

   
    users['inbound_outbound_ratio'] = users['inbound_transactions'] / (users['inbound_transactions'] + users['outbound_transactions'])

    users['completed_transactions'] = users['completed_transactions'] / trans_per_user['TransPerUser']
    users['declined_transactions'] = users['declined_transactions'] / trans_per_user['TransPerUser']
    users['reverted_transactions'] = users['reverted_transactions'] / trans_per_user['TransPerUser']
    users['failed_transactions'] = users['failed_transactions'] / trans_per_user['TransPerUser']
    users['pending_transactions'] = users['pending_transactions'] / trans_per_user['TransPerUser']
    users['cancelled_transactions'] = users['cancelled_transactions'] / trans_per_user['TransPerUser']
    
    return users.fillna(0)

In [0]:
def clean_string(astring):
    #convert to upper and replace empty space
    astring = astring.replace(" ","").upper()
    #remove numbers
    clean = ''.join([i for i in astring if not i.isdigit()])
    #remove special characters
    clean = re.sub(r"^\W+", "", clean)
    #if empty rename to Other
    clean.strip()
    if len(clean) < 3 or clean == "" : clean = 'OTHER'
    #check for buchurest and prague
    if(clean == 'BUCURESTI' or clean == 'BUCUREȘTI'): clean = 'BUCHAREST'
    if(clean == 'PRAHA'): clean = 'PRAGUE'
    if(clean == 'WARSZAWA'): clean = 'WARSAW'
    if(clean == 'CO.DOUBLIN'): clean = 'DUBLIN'
    if(clean == 'LISBOA'): clean = 'LISBON'
    if(clean == 'MILANO'): clean = 'MILAN'
    if(clean == 'RĪGA'): clean = 'RIGA'
    if(clean == 'BRUXELLES'): clean = 'BRUSSELS'
    return clean


#Giorgos
#Map countries to other countries
c2c_dict = {'MQ' : 'FR','RE' : 'FR','GP' : 'FR','GI' : 'ES','GG' : 'GB','JE' : 'GB','IM' : 'IE'}            
def c2c(country):
    if country in c2c_dict.keys():
        return c2c_dict.get(country)
    return country


#Giorgos
#adjusted net annual income per capita
dict2018 = {'GB' : 35837, 'PL':12907,'FR':34837,'IE':43843,'RO':9561,'ES':25686,
            'LT':16125,'PT':18799,'MT': 17156 ,'DE':40265,'CH':65578,'CZ':17146,'IT':28806,
            'GR':17147,'CY':26328,'LV':13726,'NL' : 44668,'HU':12896,'BE':38814,'SE':46002,'BG':8073,
            'DK':51867,'NO':66019,'SI':20691,'AT':42074,'SK':15059,
            'HR':12586,'FI':41120,'EE':18993,'LU':60159,'IS' : 58011,'LI': 59872,'AU' : 43346}
 
#groups =  0-20k,20k-40k,40k-60k,above 60k
 
def group_countries(adict):
    for key,value in adict.items():
        if value < 20000:
            adict[key] =  1
        elif value >=20000 and value <40000:
            adict[key] = 2
        elif value < 60000 and  value >= 40000:
            adict[key] = 3
        else:
            adict[key] =4
    return adict


#Giorgos
#Map cities to 'IsCapital'
capitals = ['LONDON','WARSAW','PARIS','DUBLIN','BUCHAREST','MADRID','VILNIUS','LISBON',
           'VALLETTA','BERLIN','BERN','PRAGUE','MILAN','ATHENS','NICOSIA','RIGA','AMSTERDAM',
           'BUDAPEST','BRUSSELS','STOCKHOLM','SOFIA','COPENHAGEN','OSLO','LJUBLJANA','VIENNA',
            'BRATISLAVA','ZAGREB','HELSINSKI','TALLINN','LUXEMBOURG','STHELIER','GIBRALTAR']
def is_capital(city):
    if city in capitals:
        return 1
    return 0


def extract_nots_features(users, notifications):
    notifications = extract_date_features(notifications)
    notifications = pd.merge(notifications,users,on='user_id',how='left')


    #number of notifications per user
    nots_per_user = pd.DataFrame(notifications.groupby(['user_id'])['channel'].size().reset_index(name='NotsPerUser'))
    users = pd.merge(nots_per_user,users,on='user_id',how='outer')
    
    #number of emails per user
    emails_per_user = pd.DataFrame(notifications[notifications['channel']== 'EMAIL'].groupby(['user_id'])['channel'].size().reset_index(name='EmailsPerUser'))
    users = pd.merge(emails_per_user,users,on='user_id',how='outer')

    
    #number of push per user
    push_per_user = pd.DataFrame(notifications[notifications['channel'] == 'PUSH'].groupby('user_id')['channel'].size().reset_index(name='PushPerUser'))
    users = pd.merge(push_per_user,users,on='user_id',how='outer')
    
    #number of SMS per user
    sms_per_user = pd.DataFrame(notifications[notifications['channel'] == 'SMS'].groupby('user_id')['channel'].size().reset_index(name='SmsPerUser'))
    users = pd.merge(sms_per_user,users,on='user_id',how='outer')
    
        
    users['AvgNotsPerMonth'] = users['NotsPerUser'] / users['months_subscribed']
    users['AvgEmailsPerMonth'] = users['EmailsPerUser'] / users['months_subscribed']
    users['AvgPushPerMonth'] = users['PushPerUser'] / users['months_subscribed']
    users['AvgSmsPerMonth'] = users['SmsPerUser'] / users['months_subscribed']
    
    return users.fillna(0)


def diff_month(d2):
    return (date(2019,5,20).year - d2.year) * 12 + date.today().month - d2.month


#Group notifications reasons to : 
#1)Promotion - CustomerCentricity :
#PROMO,INVEST_IN_GOLD,PROMO_CARD_ORDER,BLACK_FRIDAY,BLUE_TUESDAY,
#SILVER_ENGAGEMENT_FEES_SAVED,METAL_RESERVE_PLAN,JOINING_ANNIVERSARY,ONBOARDING_TIPS_ACTIVATED_USERS,WELCOME_BACK

#2)Services : REENGAGEMENT_ACTIVE_FUNDS,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,LOST_CARD_ORDER,
#MADE_MONEY_REQUEST_NOT_SPLIT_BILL,ENGAGEMENT_SPLIT_BILL_RESTAURANT,SILVER_ENGAGEMENT_INACTIVE_CARD,

reasons_dict = {'PROMO':0,'INVEST_IN_GOLD':0,'PROMO_CARD_ORDER':0,'BLACK_FRIDAY':0,'BLUE_TUESDAY':0,
                'SILVER_ENGAGEMENT_FEES_SAVED':0,'METAL_RESERVE_PLAN':0,'JOINING_ANNIVERSARY':0,
                'ONBOARDING_TIPS_ACTIVATED_USERS':0,'WELCOME_BACK':0,'REENGAGEMENT_ACTIVE_FUNDS':1,'NO_INITIAL_CARD_ORDER':1,
                'NO_INITIAL_CARD_USE':1,'LOST_CARD_ORDER':1,'MADE_MONEY_REQUEST_NOT_SPLIT_BILL':1,
                'ENGAGEMENT_SPLIT_BILL_RESTAURANT':1,'SILVER_ENGAGEMENT_INACTIVE_CARD':1}

def map_reasons(astring):
    if astring in reasons_dict.keys():
        return reasons_dict.get(astring)
    return -1



def days_from_last_interaction(date):
    date = str(date)
    dt = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
    delta =  datetime.datetime(2019,5,20) - dt
    return delta.days


#Giorgos
def epoch_of_year(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Autumn'

#Giorgos
def half_of_month(day):
    if day <= 15 : return 1
    return 2


#Giorgos
def extract_date_features(dataframe):
    dataframe['created_date'] = [datetime.datetime.strptime(str(date_time_str), '%Y-%m-%d %H:%M:%S.%f') for date_time_str in dataframe['created_date']]
    dataframe['part_of_day'] = (dataframe['created_date'].dt.hour %24 + 4) //4
    '''dataframe['part_of_day'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'},inplace=True)'''
    dataframe['part_of_week'] = dataframe['created_date'].dt.weekday
    '''dataframe['part_of_week'].replace({0:'Weekday',
                                      1:'Weekday',
                                      2:'Weekday',
                                      3:'Weekday',
                                      4:'Weekday',
                                      5:'Weekend',
                                      6:'Weekend'},inplace=True)'''
    dataframe['Half_of_month'] = dataframe['created_date'].dt.day.apply(half_of_month)
    dataframe['Month'] = dataframe['created_date'].dt.month
    dataframe['Epoch'] = dataframe['created_date'].dt.month.apply(epoch_of_year).map({'Winter':1,'Spring':2,'Summer':3,'Autumn':4})
    dataframe['Year'] = dataframe['created_date'].dt.year

    return dataframe


def diff_year(year):
    return date(2019,5,20).year - year

In [25]:
drive.mount('/content/drive')
project_dir = '/content/drive/My Drive/Colab Notebooks/Data Science-Project Future/pf-ds-thes-team2/'

devices_file = project_dir + 'data/devices.csv'
users_file = project_dir + 'data/users.csv'
trans1_file = project_dir + 'data/transactions_1.csv'
trans2_file = project_dir + 'data/transactions_2.csv'
trans3_file = project_dir + 'data/transactions_3.csv'
notif_file = project_dir + 'data/notifications.csv'
country_codes_file = project_dir + 'auxiliary_data/country_codes.csv'

users = pd.read_csv(users_file)
devices = pd.read_csv(devices_file)
transactions_1 = pd.read_csv(trans1_file)
transactions_2 = pd.read_csv(trans2_file)
transactions_3 = pd.read_csv(trans3_file)
notifications = pd.read_csv(notif_file)

all_transactions = [transactions_1, transactions_2, transactions_3]
transactions = pd.concat(all_transactions)
transactions = transactions.reset_index().drop(['index'], axis=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#users = convert_country_code(users, country_codes_file)
users = user_id_to_int(users)
devices = user_id_to_int(devices)
transactions = user_id_to_int(transactions)
notifications = user_id_to_int(notifications)
transactions = cardholder_fillna(transactions)

users_trans = users.merge(transactions, on='user_id', how='left')

In [0]:
'''users = add_cardholder_presence(users, users_trans)
users = add_months_subscribed(users)
users = add_largest_trans_amount(users, users_trans)
users = add_cardholder_presence(users, users_trans)
#users = add_abroad_to_non_abroad(users, users_trans)
users = add_completed_to_failed_trans_ratio(users,users_trans)
users = add_transaction_count(users,users_trans)
users = add_average_amount_per_transaction(users,users_trans)
users = add_trans_total_amount(users,users_trans)
users = add_inbound_outbound(users,users_trans)
users = add_col_brand(users)
users = mapping_data(users) 
users = add_col_age(users)
users = add_mcc(users,users_trans)'''

users = add_months_subscribed(users)
users = mapping_data(users)
users = extract_trans_features(users, transactions)

In [0]:
users['city'] = users['city'].apply(clean_string)
users['city'] = users['city'].apply(is_capital)

users['country'] = users['country'].apply(c2c)
users['country'] = users['country'].map(group_countries(dict2018))

users['birth_year'] = [diff_year(year) for year in users['birth_year']]
users = users.rename(columns={"birth_year": "Age"})

users = extract_nots_features(users, notifications)

In [0]:
cols_to_drop = ['attributes_notifications_marketing_push', 'attributes_notifications_marketing_email']
users = users.drop(columns = cols_to_drop)

#users = users.dropna()

users = users.reset_index()
users.to_csv(project_dir + 'auxiliary_data/users_features.csv', index = False)

In [0]:
#user_temp = users

users = users[['plan', 'inbound_transactions', 'outbound_transactions', 'user_settings_crypto_unlocked', 'country', 'NotsPerUser', 'AvgNotsPerMonth', 'AvgPushPerMonth', 'inbound_outbound_ratio']]

In [0]:
users['inbound_transactions'] = remove_outliers(users['inbound_transactions'])
users['inbound_transactions'] = normalize(users['inbound_transactions'])

users['NotsPerUser'] = remove_outliers(users['NotsPerUser'])
users['NotsPerUser'] = normalize(users['NotsPerUser'])

users['outbound_transactions'] = remove_outliers(users['outbound_transactions'])
users['outbound_transactions'] = normalize(users['outbound_transactions'])

users['AvgNotsPerMonth'] = standardize(users['AvgNotsPerMonth'])
users['AvgPushPerMonth'] = standardize(users['AvgPushPerMonth'])
users['inbound_outbound_ratio'] = standardize(users['inbound_outbound_ratio'])

In [0]:
users.to_csv(project_dir + 'auxiliary_data/users_selected_features.csv', index = False)